In [1]:
#### 26th March,2024

In [2]:
import pandas as pd 


path1=('HK1_overlap_bed_df-2 copy.xlsx')
colnames1=('chrom','start','end','TAD_id','Chr.','chr. start','chr. stop','sample ID','virus type')
df1=pd.read_excel(path1 , names=colnames1)
df1


/tmp/ipykernel_37267/1381604382.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


chrom     start       end   TAD_id  Chr.  chr. start  chr. stop  \
0      13  73360000  74290000  TAD3327    13    73619902   73619903   
1      13  73360000  74290000  TAD3327    13    73620050   73620051   
2      13  73360000  74290000  TAD3327    13    73620192   73620193   
3      13  73360000  74290000  TAD3327    13    73621338   73621339   
4      13  73360000  74290000  TAD3327    13    73625995   73625996   
..    ...       ...       ...      ...   ...         ...        ...   
63      9   5160000   5610000  TAD6251     9     5473811    5473812   
64      9   5160000   5610000  TAD6251     9     5473811    5473812   
65      9   5160000   5610000  TAD6251     9     5489910    5489911   
66      9   5160000   5610000  TAD6251     9     5542796    5542797   
67      9   5160000   5610000  TAD6251     9     5544852    5544853   

       sample ID virus type  
0   TCGA-CN-4741      HPV16  
1   TCGA-CN-4741      HPV16  
2   TCGA-CN-4741      HPV16  
3   TCGA-CN-4741      HPV16  
4   TCGA-CN-4741      HPV16  
..           ...        ...  
63  TCGA-CV-5443      HPV16  
64  TCGA-CV-5443      HPV16  
65       GS18088      HPV16  
66       GS21036      HPV16  
67       GS21036      HPV16  

[68 rows x 9 columns]

In [3]:
path2=('CGS.xlsx')
colnames2=('Genome Location','Gene Symbol','Role in Cancer')
df2=pd.read_excel(path2,names=colnames2)
df2[['chrom', 'interval']] = df2['Genome Location'].str.split(':', expand=True)
df2[['start','stop']] = df2['interval'].str.split('-', expand=True)
df2
df3=df2[['chrom', 'start', 'stop','Gene Symbol', 'Role in Cancer']]
df3
##df1
#df2

chrom      start       stop Gene Symbol         Role in Cancer
0       1  110338506  110346677       RBM15                 fusion
1       1   11106535   11262507        MTOR               oncogene
2       1  114392777  114511160      TRIM33            TSG, fusion
3       1  114704469  114716894        NRAS               oncogene
4       1  116373867  116404772      ATP1A1          oncogene, TSG
..    ...        ...        ...         ...                    ...
738     X   71096197   71103535       FOXO4  oncogene, TSG, fusion
739     X   71118723   71142447       MED12                    TSG
740     X   71239624   71254140       ZMYM3                    TSG
741     X   71283583   71301168        NONO                 fusion
742     X   77504878   77786269        ATRX                    TSG

[743 rows x 5 columns]

In [4]:
df3['Gene Symbol'].nunique()

743

In [5]:
pip install pyliftover


Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from pyliftover import LiftOver 

# Assuming df3 is your DataFrame

# Ensure chromosome names are strings and formatted correctly
df3['chrom'] = df3['chrom'].astype(str).apply(lambda x: 'chr' + x if not x.startswith('chr') else x)

# Ensure start and stop positions are numeric
df3['start'] = pd.to_numeric(df3['start'], errors='coerce')
df3['stop'] = pd.to_numeric(df3['stop'], errors='coerce')

# Drop rows with NaN values in 'start' or 'stop' after conversion attempt
df3.dropna(subset=['start', 'stop'], inplace=True)

# Create a LiftOver object for HG38 to HG19 conversion
lo = LiftOver('hg38', 'hg19')

def convert_coordinates(row):
    start_conversion = lo.convert_coordinate(row['chrom'], int(row['start']))
    stop_conversion = lo.convert_coordinate(row['chrom'], int(row['stop']))
    
    if start_conversion and stop_conversion:
        row['start'] = start_conversion[0][1]
        row['stop'] = stop_conversion[0][1]
    else:
        print(f"Conversion failed for gene {row['Gene Symbol']}")
    return row

# Apply the conversion to each row
df3= df3.apply(convert_coordinates, axis=1)

# Display the updated DataFrame
print()


/tmp/ipykernel_37267/2435752176.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['chrom'] = df3['chrom'].astype(str).apply(lambda x: 'chr' + x if not x.startswith('chr') else x)
/tmp/ipykernel_37267/2435752176.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['start'] = pd.to_numeric(df3['start'], errors='coerce')
/tmp/ipykernel_37267/2435752176.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Conversion failed for gene CRLF2



In [7]:
# Convert 'start' and 'stop' to numeric, coercing errors to NaN
df3['start'] = pd.to_numeric(df3['start'], errors='coerce')
df3['stop'] = pd.to_numeric(df3['stop'], errors='coerce')

# Now, drop rows with NaN in 'start' or 'stop'
df3 = df3.dropna(subset=['start', 'stop'])
df3
# Optionally, reset the index if you plan to iterate over the DataFrame
df3.reset_index(drop=True, inplace=True)
df3

chrom        start         stop Gene Symbol         Role in Cancer
0    chr1  110881128.0  110889299.0       RBM15                 fusion
1    chr1   11166592.0   11322564.0        MTOR               oncogene
2    chr1  114935399.0  115053781.0      TRIM33            TSG, fusion
3    chr1  115247090.0  115259515.0        NRAS               oncogene
4    chr1  116916489.0  116947394.0      ATP1A1          oncogene, TSG
..    ...          ...          ...         ...                    ...
732  chrX   70316047.0   70323385.0       FOXO4  oncogene, TSG, fusion
733  chrX   70338573.0   70362297.0       MED12                    TSG
734  chrX   70459474.0   70473990.0       ZMYM3                    TSG
735  chrX   70503433.0   70521018.0        NONO                 fusion
736  chrX   76760356.0   77041755.0        ATRX                    TSG

[737 rows x 5 columns]

In [8]:
# Convert 'start' and 'stop' to integers
df3['start'] = df3['start'].astype(int)
df3['stop'] = df3['stop'].astype(int)
df3

chrom      start       stop Gene Symbol         Role in Cancer
0    chr1  110881128  110889299       RBM15                 fusion
1    chr1   11166592   11322564        MTOR               oncogene
2    chr1  114935399  115053781      TRIM33            TSG, fusion
3    chr1  115247090  115259515        NRAS               oncogene
4    chr1  116916489  116947394      ATP1A1          oncogene, TSG
..    ...        ...        ...         ...                    ...
732  chrX   70316047   70323385       FOXO4  oncogene, TSG, fusion
733  chrX   70338573   70362297       MED12                    TSG
734  chrX   70459474   70473990       ZMYM3                    TSG
735  chrX   70503433   70521018        NONO                 fusion
736  chrX   76760356   77041755        ATRX                    TSG

[737 rows x 5 columns]

In [9]:
# Find rows where the start position is greater than or equal to the end position
invalid_rows = df3[df3['start'] >= df3['stop']]

# Display these rows to understand the issue
print(invalid_rows)

    chrom      start       stop Gene Symbol Role in Cancer
9    chr1  144995020  144851425     PDE4DIP         fusion
83  chr10   51589408   51565213       NCOA4    TSG, fusion


In [10]:
# Assuming df3 is your original DataFrame

# Drop the row with index 9
df3_dropped = df3.drop(index=[9,83])

# Reset the index and assign the result back to df3_dropped
df3_dropped = df3_dropped.reset_index(drop=True)

# Display the first 10 rows of the modified DataFrame
df3_dropped

chrom      start       stop Gene Symbol         Role in Cancer
0    chr1  110881128  110889299       RBM15                 fusion
1    chr1   11166592   11322564        MTOR               oncogene
2    chr1  114935399  115053781      TRIM33            TSG, fusion
3    chr1  115247090  115259515        NRAS               oncogene
4    chr1  116916489  116947394      ATP1A1          oncogene, TSG
..    ...        ...        ...         ...                    ...
730  chrX   70316047   70323385       FOXO4  oncogene, TSG, fusion
731  chrX   70338573   70362297       MED12                    TSG
732  chrX   70459474   70473990       ZMYM3                    TSG
733  chrX   70503433   70521018        NONO                 fusion
734  chrX   76760356   77041755        ATRX                    TSG

[735 rows x 5 columns]

In [11]:
# Function to standardize chromosome names
def standardize_chr_name(chr_name):
    # Ensure the chromosome name is a string
    chr_name_str = str(chr_name).lower()
    
    # Remove 'chr' prefix and handle specific cases
    if chr_name_str.startswith('chr'):
        chr_name_clean = chr_name_str.replace('chr', '')
        if chr_name_clean == 'x':
            return 'X'  # Standardize to uppercase 'X' for the sex chromosome
        else:
            return chr_name_clean.upper()  # Convert back to uppercase for numerical chromosomes
    else:
        return chr_name_str.upper()  # Return the uppercase version of the original name

# Apply the standardization function to the 'chr' column
df3_dropped['chrom'] = df3_dropped['chrom'].apply(standardize_chr_name)


In [12]:
import pandas as pd
import pybedtools
from pybedtools import BedTool
### sorting dataframes 
bed1=BedTool.from_dataframe(df1)
bed3=BedTool.from_dataframe(df3_dropped)
sorted_bed1= bed1
sorted_bed3= bed3.sort()


####m merging sorted files 
merged_sorted_bed1=sorted_bed1.merge()
merged_sorted_bed3=sorted_bed3
print(merged_sorted_bed1.head())
print(merged_sorted_bed3.head())

13	73360000	74290000
 3	181430000	181970000
 3	188950000	189660000
 8	127830000	128740000
 9	5160000	5610000
 None
1	2160134	2241558	SKI	oncogene
 1	2487805	2495268	TNFRSF14	TSG
 1	2985775	3355185	PRDM16	oncogene, fusion
 1	6245080	6259672	RPL22	TSG, fusion
 1	6845384	7829766	CAMTA1	TSG, fusion
 1	11166592	11322564	MTOR	oncogene
 1	14031350	14151574	PRDM2	TSG
 1	15818765	15850790	CASP9	TSG
 1	16174359	16266955	SPEN	TSG
 1	17345217	17380665	SDHB	TSG
 None


In [13]:
#### intersecting TAD coor with CGS dataset

tad_cgs_bed=merged_sorted_bed1.intersect(merged_sorted_bed3, wa=True , wb=True)
tad_cgs_bed_df=tad_cgs_bed.to_dataframe()
tad_cgs_bed_df
tad_cgs_bed_df

chrom      start        end  name      score     strand thickStart  \
0      3  181430000  181970000     3  181429712  181432224       SOX2   
1      3  188950000  189660000     3  189349216  189615068       TP63   
2      9    5160000    5610000     9    5450525    5470547      CD274   
3      9    5160000    5610000     9    5510570    5571254   PDCD1LG2   

           thickEnd  
0          oncogene  
1     oncogene, TSG  
2       TSG, fusion  
3  oncogene, fusion

In [14]:
tad_cgs_bed_df.columns=('chrom-TAD','TAD_start','TAD_stop','chrom','start','stop','Genes','Type')

In [17]:
tad_cgs_bed_df

chrom-TAD  TAD_start   TAD_stop  chrom      start       stop     Genes  \
0          3  181430000  181970000      3  181429712  181432224      SOX2   
1          3  188950000  189660000      3  189349216  189615068      TP63   
2          9    5160000    5610000      9    5450525    5470547     CD274   
3          9    5160000    5610000      9    5510570    5571254  PDCD1LG2   

               Type  
0          oncogene  
1     oncogene, TSG  
2       TSG, fusion  
3  oncogene, fusion

In [18]:
tad_cgs_bed_df.to_excel('tad_cgs_HK1.xlsx')

In [19]:
pybedtools.cleanup()